In [ ]:
require 'nn'
require 'rnn'
require 'optim'

In [ ]:
nepochs = 250
learning_rate = 0.1
gamma =0.2
cuts =4
base_explore_rate = 0.1
delta = 1./(nepochs/cuts) 

SKIP = 1
SELECT = 2

vocabSize = 16
embeddingSize = 64

torch.manualSeed(420)
math.randomseed(420)

sentenceLookup = nn.Sequential():add(
    nn.LookupTableMaskZero(vocabSize, embeddingSize)):add(
    nn.Sum(2, 3, false))

queryLookup = sentenceLookup:clone() --"weight", "gradWeight")
summaryLookup = sentenceLookup:clone() --"weight", "gradWeight")

model = nn.Sequential():add(
    nn.ParallelTable():add(
        sentenceLookup):add(
        queryLookup):add(
        summaryLookup)):add(
    nn.JoinTable(2)):add(
    nn.Tanh()):add(
    nn.Linear(embeddingSize * 3, 2)) --:add(
    --nn.Tanh()):add(
    --nn.Linear(embeddingSize, 2))
criterion = nn.MSECriterion()
params, gradParams = model:getParameters()

function buildSummary(actions, sentences, buffer)
    buffer:zero()

     bufferSize = buffer:size(2)
     actionsSize = actions:size(1)
     sentencesSize = sentences:size(2)

     mask1 = torch.eq(actions:select(2,2), 1):view(actionsSize, 1):expand(
        actionsSize, sentencesSize)
     allTokens = sentences:maskedSelect(mask1)
     mask2 = torch.gt(allTokens,0)
     allTokens = allTokens:maskedSelect(mask2)

    if allTokens:dim() > 0 then
         copySize = math.min(bufferSize, allTokens:size(1))

        buffer[1]:narrow(1, bufferSize - copySize + 1, copySize):copy(
            allTokens:narrow(1, allTokens:size(1) - copySize + 1, copySize))
    end
    return buffer
end

function buildTokenCounts(summary)
    counts = {}
    for i=1,summary:size(2) do
        if summary[1][i] > 0 then
             token = summary[1][i]
            if counts[token] == nil then
                counts[token] = 1
            else
                counts[token] = counts[token] + 1
            end
        end
    end
    return counts
end

function rougeScores(genSummary, refSummary)
     genTotal = 0
     refTotal = 0
     intersection = 0
    for k, refCount in pairs(refSummary) do
         genCount = genSummary[k]
        if genCount == nil then genCount = 0 end
        intersection = intersection + math.min(refCount, genCount)
        refTotal = refTotal + refCount
    end
    for k,genCount in pairs(genSummary) do
        genTotal = genTotal + genCount
    end

    if genTotal == 0 then 
        genTotal = 1 
    end
     recall = intersection / refTotal
     prec = intersection / genTotal
    if recall > 0 and prec > 0 then
        f1 = 2 * recall * prec / (recall + prec)
    else 
        f1 = 0
    end
    return recall, prec, f1
end

In [ ]:
optimParams = {
    learningRate = learning_rate,
}

maxSummarySize = 36
epsilon = 1.0
query = torch.LongTensor{{0, 1, 4, 3}}
sentenceStream = torch.Tensor{{0, 1, 3, 4}, 
                                {7, 6, 5 ,8}, 
                                {0, 2, 4, 3}, 
                                {7, 5, 8, 6}, 
                                {1, 4, 3, 2}, 
                                {13, 14, 15, 16}}

refSummary = torch.Tensor{{1,3,4,2,4,3,1,4,3,2,9,10,12,11}}
refCounts = buildTokenCounts(refSummary)


streamSize = sentenceStream:size(1)
bestActions = torch.ByteTensor{{0,1},{1,0},{0,1},{1,0},{0,1},{1,0}}


buffer = torch.Tensor(1, maxSummarySize):zero()
bestSummary = buildSummary(
    bestActions:narrow(1, 1, 6), 
    sentenceStream:narrow(1, 1, 6),
    buffer:narrow(1, 1, 1)
    )

generatedCounts = buildTokenCounts(bestSummary) 
bestrecall, bestprec, bestf1 = rougeScores(generatedCounts, refCounts)
print(string.format("TRUE {RECALL = %.6f, PREC = %.6f, F1 = %.6f}", bestrecall, bestprec, bestf1))

In [ ]:
epoch = 1

In [ ]:
actions = torch.ByteTensor(streamSize,2):fill(0)
exploreDraws = torch.Tensor(streamSize)
summaryBuffer = torch.LongTensor(streamSize + 1, maxSummarySize):zero()
qValues = torch.Tensor(streamSize, 2):zero()
rouge = torch.Tensor(streamSize + 1):zero()
rougebp = torch.Tensor(streamSize + 1):zero()

rouge[1] = 0
exploreDraws:uniform(0, 1)
summary = summaryBuffer:zero():narrow(1,1,1)

In [ ]:
for i=1, streamSize do
    sentence = sentenceStream:narrow(1, i, 1)
    qValues[i]:copy(model:forward({sentence, query, summary}))
    
    if exploreDraws[i] <= epsilon then
        actions[i][torch.random(SKIP, SELECT)] = 1
    else 
        if qValues[i][SKIP] > qValues[i][SELECT] then
            actions[i][SKIP] = 1
        else
            actions[i][SELECT] = 1
        end
    end
    summary = buildSummary(
        actions:narrow(1, 1, i), 
        sentenceStream:narrow(1, 1, i),
        summaryBuffer:narrow(1, i + 1, 1)
        )

    generatedCounts = buildTokenCounts(summary) 
    recall, prec, f1 = rougeScores(generatedCounts, refCounts)
    rouge[i + 1] = f1
    rougebp[i] = f1
end

max, argmax = torch.max(qValues, 2)
rewards0 = rouge:narrow(1,2, streamSize) - rouge:narrow(1,1, streamSize) 
rewards = rewards0 + gamma * rewards0:narrow(1, 2, streamSize-1):resize(streamSize)

querySize = query:size(2)
summaryBatch = summaryBuffer:narrow(1, 1, streamSize)
queryBatch = query:view(1, querySize):expand(streamSize, querySize) 

input = {sentenceStream, queryBatch, summaryBatch}

In [ ]:
function feval(params)
    gradParams:zero()
     predQ = model:forward(input)
     maskLayer = nn.MaskedSelect()
     predQOnActions = maskLayer:forward({predQ, actions})
     loss = criterion:forward(predQOnActions, reward)
     gradOutput = criterion(predQOnActions, reward)
     gradMaskLayer = maskLayer:backward({predQ, actions}, gradOutput)
    model:backward(input, gradMaskLayer[1])
    return loss, gradParams    
end

 _, loss = optim.adam(feval, params, optimParams)

In [ ]:
out = string.format("%i; %.3f;%.6f;%.6f; {min=%.3f, max=%.3f}; {min=%.3f, max=%.3f}; {%i,%i,%i,%i,%i,%i,%i,%i,%i,%i,%i,%i}\n", 
    epoch, epsilon, loss[1], rouge[streamSize + 1],
    reward:min(), reward:max(),
    qValues:min(), qValues:max(),
    actions[1][1], 
    actions[1][2], 
    actions[2][1], 
    actions[2][2], 
    actions[3][1], 
    actions[3][2], 
    actions[4][1],
    actions[4][2],
    actions[5][1], 
    actions[5][2], 
    actions[6][1], 
    actions[6][2] 
    )


In [ ]:
out

In [ ]:
nepochs

# Trying this separately

In [65]:
require 'nn'
require 'rnn'
require 'optim'

In [84]:
nepochs = 1000
learning_rate = 1e-6
gamma =0.2
cuts =4
base_explore_rate = 0.1 
delta = 1./(nepochs/cuts) 

SKIP = 1
SELECT = 2

vocabSize = 16
embeddingSize = 64

torch.manualSeed(420)
math.randomseed(420)

sentenceLookup = nn.Sequential():add(
    nn.LookupTableMaskZero(vocabSize, embeddingSize)):add(
    nn.Sum(2, 3, false))

queryLookup = sentenceLookup:clone() --"weight", "gradWeight")
summaryLookup = sentenceLookup:clone() --"weight", "gradWeight")

model = nn.Sequential():add(
    nn.ParallelTable():add(
        sentenceLookup):add(
        queryLookup):add(
        summaryLookup)):add(
    nn.JoinTable(2)):add(
    nn.Tanh()):add(
    nn.Linear(embeddingSize * 3, 2)) --:add(
    --nn.Tanh()):add(
    --nn.Linear(embeddingSize, 2))
criterion = nn.MSECriterion()
params, gradParams = model:getParameters()

function buildSummary(actions, sentences, buffer)
    buffer:zero()

     bufferSize = buffer:size(2)
     actionsSize = actions:size(1)
     sentencesSize = sentences:size(2)

     mask1 = torch.eq(actions:select(2,2), 1):view(actionsSize, 1):expand(
        actionsSize, sentencesSize)
     allTokens = sentences:maskedSelect(mask1)
     mask2 = torch.gt(allTokens,0)
     allTokens = allTokens:maskedSelect(mask2)

    if allTokens:dim() > 0 then
         copySize = math.min(bufferSize, allTokens:size(1))

        buffer[1]:narrow(1, bufferSize - copySize + 1, copySize):copy(
            allTokens:narrow(1, allTokens:size(1) - copySize + 1, copySize))
    end
    return buffer
end

function buildTokenCounts(summary)
    counts = {}
    for i=1,summary:size(2) do
        if summary[1][i] > 0 then
             token = summary[1][i]
            if counts[token] == nil then
                counts[token] = 1
            else
                counts[token] = counts[token] + 1
            end
        end
    end
    return counts
end

function rougeScores(genSummary, refSummary)
     genTotal = 0
     refTotal = 0
     intersection = 0
    for k, refCount in pairs(refSummary) do
         genCount = genSummary[k]
        if genCount == nil then genCount = 0 end
        intersection = intersection + math.min(refCount, genCount)
        refTotal = refTotal + refCount
    end
    for k,genCount in pairs(genSummary) do
        genTotal = genTotal + genCount
    end

    if genTotal == 0 then 
        genTotal = 1 
    end
     recall = intersection / refTotal
     prec = intersection / genTotal
    if recall > 0 and prec > 0 then
        f1 = 2 * recall * prec / (recall + prec)
    else 
        f1 = 0
    end
    return recall, prec, f1
end

In [85]:
optimParams = {
    learningRate = learning_rate,
}

maxSummarySize = 36
epsilon = 1.0
query = torch.LongTensor{{0, 1, 4, 3}}
sentenceStream = torch.Tensor{{0, 1, 3, 4}, 
                                {7, 6, 5 ,8}, 
                                {0, 2, 4, 3}, 
                                {7, 5, 8, 6}, 
                                {1, 4, 3, 2}, 
                                {13, 14, 15, 16}}

refSummary = torch.Tensor{{1,3,4,2,4,3,1,4,3,2,9,10,12,11}}
refCounts = buildTokenCounts(refSummary)


streamSize = sentenceStream:size(1)
bestActions = torch.ByteTensor{{0,1},{1,0},{0,1},{1,0},{0,1},{1,0}}


buffer = torch.Tensor(1, maxSummarySize):zero()
bestSummary = buildSummary(
    bestActions:narrow(1, 1, 6), 
    sentenceStream:narrow(1, 1, 6),
    buffer:narrow(1, 1, 1)
    )

generatedCounts = buildTokenCounts(bestSummary) 
bestrecall, bestprec, bestf1 = rougeScores(generatedCounts, refCounts)
print(string.format("TRUE {RECALL = %.6f, PREC = %.6f, F1 = %.6f}", bestrecall, bestprec, bestf1))

TRUE {RECALL = 0.714286, PREC = 1.000000, F1 = 0.833333}	


In [86]:
tmp0 = {}
tmp1 = {}
tmp2 = {}
for epoch=1,nepochs do
    actions = torch.ByteTensor(streamSize,2):fill(0)
    exploreDraws = torch.Tensor(streamSize)
    summaryBuffer = torch.LongTensor(streamSize + 1, maxSummarySize):zero()
    qValues = torch.Tensor(streamSize, 2):zero()
    rouge = torch.Tensor(streamSize + 1):zero()

    rouge[1] = 1
    exploreDraws:uniform(0, 1)

    summary = summaryBuffer:zero():narrow(1,1,1)
    for i=1, streamSize do
        --- the i extracts individual sentences from the stream
         sentence = sentenceStream:narrow(1, i, 1)
        qValues[i]:copy(model:forward({sentence, query, summary}))

        if exploreDraws[i] <= epsilon then
            actions[i][torch.random(SKIP, SELECT)] = 1
        else 
            if qValues[i][SKIP] > qValues[i][SELECT] then
                actions[i][SKIP] = 1
            else
                actions[i][SELECT] = 1
            end
        end

        summary = buildSummary(
            actions:narrow(1, 1, i), 
            sentenceStream:narrow(1, 1, i),
            summaryBuffer:narrow(1, i + 1, 1)
            )

         generatedCounts = buildTokenCounts(summary) 
         recall, prec, f1 = rougeScores(generatedCounts, refCounts)
        rouge[i + 1] = f1
    end

     max, argmax = torch.max(qValues, 2)
    
     reward0 = rouge:narrow(1,2, streamSize) - rouge:narrow(1,1, streamSize)
     reward_tp1 = gamma * reward0:narrow(1, 2, streamSize - 1):resize(streamSize)
     reward_tp1[reward_tp1:ne(reward_tp1)] = 0
     reward = reward0 + reward_tp1
    
     tmp0[epoch] = reward0
     tmp1[epoch] = reward_tp1
     tmp2[epoch] = reward
    
     querySize = query:size(2)
     summaryBatch = summaryBuffer:narrow(1, 1, streamSize)
     queryBatch = query:view(1, querySize):expand(streamSize, querySize) 

     input = {sentenceStream, queryBatch, summaryBatch}
    if epoch == 1 then
        print(input)
    end

     function feval(params)
        gradParams:zero()
         predQ = model:forward(input)
         maskLayer = nn.MaskedSelect()
         predQOnActions = maskLayer:forward({predQ, actions})

         loss = criterion:forward(predQOnActions, reward)
         gradOutput = criterion(predQOnActions, reward)
         gradMaskLayer = maskLayer:backward({predQ, actions}, gradOutput)
        model:backward(input, gradMaskLayer[1])
        return loss, gradParams    
    end

     _, loss = optim.adam(feval, params, optimParams)
    out = string.format("%i; %.3f;%.6f;%.6f; {min=%.3f, max=%.3f}; {min=%.3f, max=%.3f}; {%i,%i,%i,%i,%i,%i,%i,%i,%i,%i,%i,%i}\n", 
            epoch, epsilon, loss[1], rouge[streamSize + 1],
            reward:min(), reward:max(),
            qValues:min(), qValues:max(),
            actions[1][1], 
            actions[1][2], 
            actions[2][1], 
            actions[2][2], 
            actions[3][1], 
            actions[3][2], 
            actions[4][1],
            actions[4][2],
            actions[5][1], 
            actions[5][2], 
            actions[6][1], 
            actions[6][2] 
        )
    print(out)

    if (epsilon - delta) <= base_explore_rate then
        epsilon = base_explore_rate
    else 
        epsilon = epsilon - delta
    end
end


{
  1 : DoubleTensor - size: 6x4
  2 : LongTensor - size: 6x4
  3 : LongTensor - size: 6x36
}


1; 1.000;0.128764;0.240000; {min=-1.000, max=0.600}; {min=-0.399, max=0.188}; {1,0,0,1,0,1,0,1,1,0,1,0}
	


2; 0.996;0.134296;0.424242; {min=-0.661, max=0.231}; {min=-0.458, max=0.455}; {0,1,0,1,1,0,0,1,0,1,0,1}
	


3; 0.992;0.098293;0.666667; {min=-1.000, max=0.353}; {min=-0.399, max=0.219}; {1,0,1,0,0,1,1,0,0,1,1,0}
	


4; 0.988;0.123964;0.625000; {min=-0.661, max=0.200}; {min=-0.399, max=0.566}; {0,1,0,1,0,1,0,1,0,1,1,0}
	


...

996; 0.100;nan;0.555556; {min=-0.661, max=0.200}; {min=nan, max=nan}; {0,1,0,1,0,1,0,1,0,1,0,1}
	


997; 0.100;nan;0.555556; {min=-0.661, max=0.200}; {min=nan, max=nan}; {0,1,0,1,0,1,0,1,0,1,0,1}
	


998; 0.100;nan;0.555556; {min=-0.661, max=0.200}; {min=nan, max=nan}; {0,1,0,1,0,1,0,1,0,1,0,1}
	


999; 0.100;nan;0.482759; {min=-1.000, max=0.277}; {min=nan, max=nan}; {1,0,0,1,0,1,0,1,0,1,1,0}
	


1000; 0.100;nan;0.555556; {min=-0.661, max=0.200}; {min=nan, max=nan}; {0,1,0,1,0,1,0,1,0,1,0,1}
	


In [133]:
torch.clamp(tmp0[360], -0.7, 0.7)

-0.6471
 0.0000
 0.0000
 0.0000
 0.0000
 0.0000
[torch.DoubleTensor of size 6]



In [104]:
tmp1[360]

 0.0000e+00
 0.0000e+00
 0.0000e+00
 0.0000e+00
 0.0000e+00
-6.2207e+230
[torch.DoubleTensor of size 6]



In [126]:
tmp1[1]

 0.0000
 0.0571
-0.0091
 0.0000
 0.0000
 0.6000
[torch.DoubleTensor of size 6]



In [129]:
nn.Threshold(0.06, 0):forward(tmp1[1])

 0.0000
 0.0000
 0.0000
 0.0000
 0.0000
 0.6000
[torch.DoubleTensor of size 6]



In [111]:
nn.Threshold(-.05, 0):forward(tmp0[360])

 0
 0
 0
 0
 0
 0
[torch.DoubleTensor of size 6]



In [101]:
tmp1[360]

 0.0000e+00
 0.0000e+00
 0.0000e+00
 0.0000e+00
 0.0000e+00
-6.2207e+230
[torch.DoubleTensor of size 6]



In [89]:
tmp2[360]

 -6.4706e-01
  0.0000e+00
  0.0000e+00
  0.0000e+00
  0.0000e+00
-6.2207e+230
[torch.DoubleTensor of size 6]



In [83]:
tmp1[6][tmp1[6]:ne(tmp1[6])] = 0

In [78]:
tmp0[6]:narrow(1, 2, streamSize-1):resize(streamSize)

0.000000
0.000000
0.000000
0.444444
0.000000
nan
[torch.DoubleTensor of size 6]



In [76]:
tmp0[6]:narrow(1, 2, streamSize-1):resize(streamSize)

0.000000
0.000000
0.000000
0.444444
0.000000
nan
[torch.DoubleTensor of size 6]



In [75]:
tmp0[6] + gamma * tmp0[6]:narrow(1, 2, streamSize-1):resize(streamSize)

-1.000000
0.000000
0.000000
0.088889
0.444444
nan
[torch.DoubleTensor of size 6]



In [72]:
tmp1[6]

0.000000
0.000000
0.000000
0.088889
0.000000
nan
[torch.DoubleTensor of size 6]



In [62]:
tmp0[242]

-0.6471
-0.0672
 0.2143
-0.0714
 0.1964
-0.0694
[torch.DoubleTensor of size 6]



In [63]:
--- If tmp0[242] < thresh, out else tmp0[242]
thresh = -0.7
out = 99
print(nn.Threshold(thresh, out):forward(tmp0[242]))

-0.6471
-0.0672
 0.2143
-0.0714
 0.1964
-0.0694
[torch.DoubleTensor of size 6]



In [64]:
--- If tmp0[242] < thresh, out else tmp0[242]
thresh = -0.25
out = 99
print(nn.Threshold(thresh, out):forward(-tmp0[242]))

 0.6471
 0.0672
-0.2143
 0.0714
-0.1964
 0.0694
[torch.DoubleTensor of size 6]



In [59]:
nn.Threshold(0.01, -0.001):forward(tmp0[242])

-0.0010
-0.0010
 0.2143
-0.0010
 0.1964
-0.0010
[torch.DoubleTensor of size 6]



In [60]:
nn.Threshold(0.1, 0.01):forward(nn.Threshold(-0.01, -0.01):forward(tmp0[242]))

 0.0100
 0.0100
 0.2143
 0.0100
 0.1964
 0.0100
[torch.DoubleTensor of size 6]



In [61]:
for i = 1, 500 do
    print(tmp0[i] + tmp1[i])
end

-1.0000
 0.0571
 0.2766
-0.0457
 0.0000
 0.0000
[torch.DoubleTensor of size 6]

-0.6605
-0.0672
-0.0091
 0.0028
 0.2311
-0.0585
[torch.DoubleTensor of size 6]

-1.0000
 0.0706
 0.3529
 0.0627
 0.3137
 0.0000
[torch.DoubleTensor of size 6]

-0.6605
-0.0244
 0.2000
-0.0321
 0.1964
 0.0000
[torch.DoubleTensor of size 6]

-1.0000
 0.0706
 0.3395
-0.0124
 0.2588
-0.0772
[torch.DoubleTensor of size 6]

-1.0000
 0.0000
 0.0000
 0.0889
 0.4444
 0.0000
[torch.DoubleTensor of size 6]

-0.6605
-0.0672
 0.0000
 0.0549
 0.2743
 0.0000
[torch.DoubleTensor of size 6]

-1
 0
 0
 0
 0
 0
[torch.DoubleTensor of size 6]

-0.6471
 0.0494
 0.2471
 0.0467
 0.2095
-0.1190
[torch.DoubleTensor of size 6]

-1.0000
 0.0571
 0.2857
 0.0549
 0.2743
 0.0000
[torch.DoubleTensor of size 6]

-1.0000
 0.0000
 0.0000
 0.0727
 0.3636
 0.0000
[torch.DoubleTensor of size 6]

-1.0000
 0.0000
 0.0000
 0.0727
 0.3636
 0.0000
[torch.DoubleTensor of size 6]

-0.6605
-0.0672
 0.0000
 0.0549
 0.2588
-0.0772
[torch.DoubleTensor of

-1
 0
 0
 0
 0
 0
[torch.DoubleTensor of size 6]

-1
 0
 0
 0
 0
 0
[torch.DoubleTensor of size 6]

-1.0000
 0.0706
 0.3395
-0.0672
 0.0000
 0.0000
[torch.DoubleTensor of size 6]

-0.6471
 0.0494
 0.2271
-0.0571
 0.2143
 0.0000
[torch.DoubleTensor of size 6]

-1.0000
 0.0706
 0.3395
-0.0672
 0.0000
 0.0000
[torch.DoubleTensor of size 6]

-0.6471
 0.0494
 0.2471
 0.0000
 0.0000
 0.0000
[torch.DoubleTensor of size 6]

-0.6471
 0.0000
 0.0000
 0.0000
-0.0134
-0.0672
[torch.DoubleTensor of size 6]

-0.6605
-0.0672
-0.0091
-0.0457
 0.0000
 0.0000
[torch.DoubleTensor of size 6]

-1
 0
 0
 0
 0
 0
[torch.DoubleTensor of size 6]

-0.6605
-0.0672
 0.0000
 0.0549
 0.2743
 0.0000
[torch.DoubleTensor of size 6]

-1
 0
 0
 0
 0
 0
[torch.DoubleTensor of size 6]

-0.6471
 0.0494
 0.2471
 0.0467
 0.2333
 0.0000
[torch.DoubleTensor of size 6]

-0.6471
 0.0494
 0.2471
 0.0000
-0.0200
-0.1000
[torch.DoubleTensor of size 6]

-0.6471
 0.0000
 0.0000
 0.0000
 0.0000
 0.0000
[torch.DoubleTensor of size 6]



6]

-0.6471
 0.0494
 0.2471
 0.0000
 0.0000
 0.0000
[torch.DoubleTensor of size 6]

-0.6471
 0.0494
 0.2471
 0.0000
 0.0000
 0.0000
[torch.DoubleTensor of size 6]

-0.6471
 0.0000
 0.0000
 0.0000
 0.0000
 0.0000
[torch.DoubleTensor of size 6]

-0.6471
 0.0494
 0.2471
 0.0467
 0.2333
 0.0000
[torch.DoubleTensor of size 6]

-0.6471
 0.0494
 0.2471
 0.0000
 0.0000
 0.0000
[torch.DoubleTensor of size 6]

-0.6605
-0.0672
 0.0000
 0.0000
 0.0000
 0.0000
[torch.DoubleTensor of size 6]

-0.6471
 0.0494
 0.2471
 0.0000
 0.0000
 0.0000
[torch.DoubleTensor of size 6]

-0.6471
 0.0494
 0.2271
-0.1000
 0.0000
 0.0000
[torch.DoubleTensor of size 6]

-0.6471
 0.0494
 0.2271
-0.1000
 0.0000
 0.0000
[torch.DoubleTensor of size 6]

-0.6471
 0.0494
 0.2471
 0.0000
 0.0000
 0.0000
[torch.DoubleTensor of size 6]

-0.6471
 0.0494
 0.2471
 0.0000
 0.0000
 0.0000
[torch.DoubleTensor of size 6]

-0.6471
 0.0494
 0.2471
 0.0000
 0.0000
 0.0000
[torch.DoubleTensor of size 6]

-0.6471
 0.0494
 0.2471
 0.0000
 0.0

.0000
[torch.DoubleTensor of size 6]

-0.6605
-0.0672
 0.0000
 0.0000
 0.0000
 0.0000
[torch.DoubleTensor of size 6]

-1.0000
 0.0000
 0.0000
 0.0000
 0.0000
 0.0000
[torch.DoubleTensor of size 6]

 -6.4706e-01
  4.9412e-02
  2.4706e-01
  4.6667e-02
  2.3333e-01
 3.4585e-323
[torch.DoubleTensor of size 6]

-0.6471
 0.0494
 0.2471
 0.0000
 0.0000
 0.0000
[torch.DoubleTensor of size 6]

-0.6471
 0.0494
 0.2271
-0.0571
 0.2143
 0.0000
[torch.DoubleTensor of size 6]

-0.6471
 0.0494
 0.2471
 0.0000
 0.0000
 0.0000
[torch.DoubleTensor of size 6]

-0.6605
-0.0672
 0.0000
 0.0000
 0.0000
 0.0000
[torch.DoubleTensor of size 6]

-0.6471
 0.0494
 0.2471
 0.0000
 0.0000
 0.0000
[torch.DoubleTensor of size 6]

-0.6471
 0.0494
 0.2471
 0.0000
 0.0000
 0.0000
[torch.DoubleTensor of size 6]

-0.6605
-0.0672
 0.0000
 0.0000
 0.0000
 0.0000
[torch.DoubleTensor of size 6]

-0.6471
 0.0494
 0.2271
-0.1000
 0.0000
 0.0000
[torch.DoubleTensor of size 6]

-0.6471
 0.0494
 0.2471
 0.0000
 0.0000
 0.0000
[tor



-0.6471
 0.0494
 0.2471
 0.0000
 0.0000
 0.0000
[torch.DoubleTensor of size 6]

-0.6471
 0.0494
 0.2471
 0.0000
 0.0000
 0.0000
[torch.DoubleTensor of size 6]

-0.6471
 0.0494
 0.2471
 0.0467
 0.2333
 0.0000
[torch.DoubleTensor of size 6]

-0.6471
 0.0494
 0.2471
 0.0467
 0.2333
 0.0000
[torch.DoubleTensor of size 6]

-0.6605
-0.0672
 0.0000
 0.0000
 0.0000
 0.0000
[torch.DoubleTensor of size 6]

-0.6471
 0.0494
 0.2471
 0.0000
 0.0000
 0.0000
[torch.DoubleTensor of size 6]

-0.6471
 0.0494
 0.2471
 0.0000
 0.0000
 0.0000
[torch.DoubleTensor of size 6]

-0.6471
 0.0494
 0.2471
 0.0000
 0.0000
 0.0000
[torch.DoubleTensor of size 6]

-0.6605
-0.0672
 0.0000
 0.0000
 0.0000
 0.0000
[torch.DoubleTensor of size 6]

 -6.4706e-01
  4.9412e-02
  2.4706e-01
  4.6667e-02
  2.3333e-01
 2.9708e-314
[torch.DoubleTensor of size 6]

-0.6471
 0.0494
 0.2471
 0.0000
 0.0000
 0.0000
[torch.DoubleTensor of size 6]

-0.6471
 0.0494
 0.2471
 0.0000
 0.0000
 0.0000
[torch.DoubleTensor of size 6]

-0.6605


 0.1825
-0.0694
[torch.DoubleTensor of size 6]

-0.6605
-0.0244
 0.2000
-0.0321
 0.1825
-0.0694
[torch.DoubleTensor of size 6]

-0.6605
-0.0672
-0.0091
 0.0028
 0.2311
-0.0585
[torch.DoubleTensor of size 6]

-0.6605
-0.0244
 0.2000
-0.0321
 0.1825
-0.0694
[torch.DoubleTensor of size 6]

-0.6605
-0.0244
 0.2000
-0.0321
 0.1825
-0.0694
[torch.DoubleTensor of size 6]

-0.6605
-0.0244
 0.2000
-0.0321
 0.1825
-0.0694
[torch.DoubleTensor of size 6]

-0.6605
-0.0244
 0.2000
-0.0321
 0.1825
-0.0694
[torch.DoubleTensor of size 6]

-0.6605
-0.0244
 0.2000
-0.0714
-0.0107
-0.0536
[torch.DoubleTensor of size 6]

-0.6605
-0.0244
 0.2143
 0.0429
 0.1964
-0.0893
[torch.DoubleTensor of size 6]

-0.6605
-0.0244
 0.2000
-0.0321
 0.1825
-0.0694
[torch.DoubleTensor of size 6]

-0.6605
-0.0244
 0.2000
-0.0321
 0.1825
-0.0694
[torch.DoubleTensor of size 6]

-0.6605
-0.0244
 0.2000
-0.0321
 0.1825
-0.0694
[torch.DoubleTensor of size 6]

-0.6471
 0.0494
 0.2271
-0.0571
 0.1964
-0.0893
[torch.DoubleTensor of s

[torch.DoubleTensor of size 6]

-0.6605
-0.0244
 0.2000
-0.0714
-0.0107
-0.0536
[torch.DoubleTensor of size 6]

-0.6605
-0.0244
 0.2000
-0.0321
 0.1825
-0.0694
[torch.DoubleTensor of size 6]

-0.6605
-0.0244
 0.2000
-0.0321
 0.1825
-0.0694
[torch.DoubleTensor of size 6]

-0.6605
-0.0244
 0.2000
-0.0321
 0.1825
-0.0694
[torch.DoubleTensor of size 6]

-0.6605
-0.0244
 0.2000
-0.0321
 0.1825
-0.0694
[torch.DoubleTensor of size 6]

-0.6605
-0.0672
-0.0091
 0.0028
 0.2311
-0.0585
[torch.DoubleTensor of size 6]

-0.6605
-0.0672
-0.0091
 0.0028
 0.2311
-0.0585
[torch.DoubleTensor of size 6]

-0.6605
-0.0244
 0.2000
-0.0321
 0.1825
-0.0694
[torch.DoubleTensor of size 6]

-0.6605
-0.0244
 0.2000
-0.0321
 0.1825
-0.0694
[torch.DoubleTensor of size 6]

-0.6605
-0.0244
 0.2000
-0.0321
 0.1825
-0.0694
[torch.DoubleTensor of size 6]

-0.6605
-0.0244
 0.2000
-0.0321
 0.1825
-0.0694
[torch.DoubleTensor of size 6]

-0.6605
-0.0244
 0.2000
-0.0321
 0.1825
-0.0694
[torch.DoubleTensor of size 6]

-0.6605


.2311
-0.0585
[torch.DoubleTensor of size 6]

-0.6605
-0.0244
 0.2000
-0.0321
 0.1825
-0.0694
[torch.DoubleTensor of size 6]

-0.6605
-0.0244
 0.2000
-0.0321
 0.1825
-0.0694
[torch.DoubleTensor of size 6]

-0.6605
-0.0244
 0.2000
-0.0714
-0.0107
-0.0536
[torch.DoubleTensor of size 6]

-0.6605
-0.0672
-0.0091
 0.0028
 0.2311
-0.0585
[torch.DoubleTensor of size 6]

-0.6605
-0.0244
 0.2000
-0.0321
 0.1825
-0.0694
[torch.DoubleTensor of size 6]

-0.6605
-0.0244
 0.2000
-0.0321
 0.1825
-0.0694
[torch.DoubleTensor of size 6]

-0.6605
-0.0244
 0.2000
-0.0321
 0.1825
-0.0694
[torch.DoubleTensor of size 6]

-0.6605
-0.0244
 0.2000
-0.0321
 0.1825
-0.0694
[torch.DoubleTensor of size 6]

-0.6605
-0.0244
 0.2000
-0.0321
 0.1825
-0.0694
[torch.DoubleTensor of size 6]

-0.6605
-0.0244
 0.2000
-0.0321
 0.1825
-0.0694
[torch.DoubleTensor of size 6]

-0.6605
-0.0244
 0.2000
-0.0321
 0.1825
-0.0694
[torch.DoubleTensor of size 6]

-0.6605
-0.0244
 0.2000
-0.0321
 0.1825
-0.0694
[torch.DoubleTensor of siz

000
-0.0321
 0.1825
-0.0694
[torch.DoubleTensor of size 6]

-0.6605
-0.0672
 0.0000
 0.0549
 0.2588
-0.0772
[torch.DoubleTensor of size 6]

-0.6605
-0.0672
-0.0091
 0.0028
 0.2311
-0.0585
[torch.DoubleTensor of size 6]

-0.6605
-0.0244
 0.2000
-0.0321
 0.1825
-0.0694
[torch.DoubleTensor of size 6]

-0.6605
-0.0244
 0.2000
-0.0321
 0.1825
-0.0694
[torch.DoubleTensor of size 6]

-0.6605
-0.0244
 0.2000
-0.0321
 0.1825
-0.0694
[torch.DoubleTensor of size 6]

-1.0000
 0.0571
 0.2766
 0.0028
 0.2311
-0.0585
[torch.DoubleTensor of size 6]

-0.6605
-0.0244
 0.2000
-0.0321
 0.1825
-0.0694
[torch.DoubleTensor of size 6]

-0.6605
-0.0244
 0.2000
-0.0321
 0.1825
-0.0694
[torch.DoubleTensor of size 6]

-0.6605
-0.0244
 0.2000
-0.0321
 0.1825
-0.0694
[torch.DoubleTensor of size 6]

-1.0000
 0.0571
 0.2766
 0.0028
 0.2311
-0.0585
[torch.DoubleTensor of size 6]

-0.6605
-0.0244
 0.2000
-0.0321
 0.1825
-0.0694
[torch.DoubleTensor of size 6]

-0.6605
-0.0244
 0.2143
 0.0429
 0.1964
-0.0893
[torch.Doubl

825
-0.0694
[torch.DoubleTensor of size 6]

-0.6605
-0.0244
 0.2000
-0.0321
 0.1825
-0.0694
[torch.DoubleTensor of size 6]

-0.6605
-0.0244
 0.2000
-0.0321
 0.1825
-0.0694
[torch.DoubleTensor of size 6]

-0.6605
-0.0244
 0.2000
-0.0321
 0.1825
-0.0694
[torch.DoubleTensor of size 6]

-0.6605
-0.0244
 0.2000
-0.0321
 0.1825
-0.0694
[torch.DoubleTensor of size 6]

-0.6605
-0.0244
 0.2000
-0.0321
 0.1825
-0.0694
[torch.DoubleTensor of size 6]

-0.6471
 0.0494
 0.2271
-0.0571
 0.1964
-0.0893
[torch.DoubleTensor of size 6]

-0.6605
-0.0244
 0.2000
-0.0321
 0.1825
-0.0694
[torch.DoubleTensor of size 6]

-0.6605
-0.0244
 0.2000
-0.0321
 0.1964
 0.0000
[torch.DoubleTensor of size 6]

-0.6605
-0.0244
 0.2000
-0.0321
 0.1825
-0.0694
[torch.DoubleTensor of size 6]

-0.6605
-0.0244
 0.2000
-0.0321
 0.1825
-0.0694
[torch.DoubleTensor of size 6]

-0.6605
-0.0244
 0.2000
-0.0321
 0.1825
-0.0694
[torch.DoubleTensor of size 6]

-0.6605
-0.0672
-0.0091
 0.0028
 0.2311
-0.0585
[torch.DoubleTensor of size 